# An analysis of the american political scene from a mediatic point of view

## Context

In the 21st century media coverage is a crucial factor for political figures. By studying the number of times a certain politician is quoted in media outlets (in our case New York Times), we can have a rough measure of how much interest does the media address to this politician. In our analysis we will study the evolution of the number of citations of some of the most important american politicians over the last few years and we will compare their evolution to the most important events in their carreer in order to see if there is any causation or correlation. We will then add some more analysis distinguishing the speakers (who quoted a certain politician) by religion, nationality and political party in order to have a better and fragmented view of the causal effects. In the end we will compare our work with Google Trends data in order to see if the conventional media outlets caption the online interest well.


## The data

We are provided with a compressed `.bz2` json file containing one row per quote. 
The `.json` has the following fields:

 - `quoteID`: Primary key of the quotation (format: "YYYY-MM-DD-{increasing int:06d}")
 - `quotation`: Text of the longest encountered original form of the quotation
 - `date`: Earliest occurrence date of any version of the quotation
 - `phase`: Corresponding phase of the data in which the quotation first occurred (A-E)
 - `probas`: Array representing the probabilities of each speaker having uttered the quotation.
      The probabilities across different occurrences of the same quotation are summed for
      each distinct candidate speaker and then normalized
      - `proba`: Probability for a given speaker
      - `speaker`: Most frequent surface form for a given speaker in the articles where the quotation occurred
 - `speaker`: Selected most likely speaker. This matches the the first speaker entry in `probas`
 - `qids`: Wikidata IDs of all aliases that match the selected speaker
 - `numOccurrences`: Number of time this quotation occurs in the articles
 - `urls`: List of links to the original articles containing the quotation 

In [4]:
# Imports we may need
import seaborn as sns
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import numpy as np
import ujson as json
import bz2

### Load Quotes and Speaker Attributes

Load quotes related to Donald Trump.

In [9]:
df_Trump = pd.read_pickle("data/df_Trump_cleaned.pkl")

df_Trump

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
12,2018-07-16-000103,[ Ensuring ] the orchestrating and timing of M...,Corey Lewandowski,[Q20740735],2018-07-16 14:05:34,2,"[[Corey Lewandowski, 0.7179], [None, 0.2754], ...",[http://www.theweek.co.uk/95082/donald-trump-s...,E
66,2018-05-09-001003,300-plus years of them cold shoulders... Obama...,Charlamagne Tha God,[Q16203002],2018-05-09 11:00:00,1,"[[Charlamagne Tha God, 0.4806], [None, 0.2924]...",[https://www.portlandmercury.com/music/2018/05...,E
212,2018-08-02-002115,A politically connected contractor made a $500...,None,[],2018-08-02 06:24:59,1,"[[None, 0.7942], [President Trump, 0.1406], [K...",[https://sunlightfoundation.com/2018/08/02/tod...,E
264,2018-08-19-001084,A Wider Danger: Trump's Troubling Attacks on J...,None,[],2018-08-19 02:05:39,3,"[[None, 0.9052], [Queen Elizabeth II, 0.0948]]","[http://www.vnews.com/Forum-Aug-19-19549635, h...",E
333,2018-09-26-004202,"After the commercial, she says, `They told me ...",None,[],2018-09-26 00:00:00,2,"[[None, 0.4595], [Tom Arnold, 0.3248], [Julie ...",[http://feeds.foxnews.com/~r/foxnews/entertain...,E
...,...,...,...,...,...,...,...,...,...
5243994,2020-02-05-103219,Trump offends and disrespects the Venezuelan p...,Jorge Arreaza,[Q6623799],2020-02-05 00:00:00,11,"[[Jorge Arreaza, 0.9164], [None, 0.0726], [Pre...",[https://www.rawstory.com/2020/02/imwithfred-t...,E
5243995,2020-02-05-103235,"Trump survived, but he is the most unpopular p...",None,[],2020-02-05 23:11:42,3,"[[None, 0.8786], [Donald Trump, 0.1214]]",[https://www.wellsvilledaily.com/zz/news/20200...,E
5243996,2020-03-13-071475,"Trump tried to mitigate the issue, saying it i...",Hassan Nasrallah,[Q181182],2020-03-13 22:15:06,1,"[[Hassan Nasrallah, 0.922], [None, 0.0741], [P...",[http://israelnationalnews.com/News/News.aspx/...,E
5243997,2020-03-15-037086,Trump's do-over approach -- he unlocked $50 bi...,Newt Gingrich,[Q182788],2020-03-15 00:00:00,40,"[[Newt Gingrich, 0.5146], [None, 0.3958], [Don...",[http://uspolitics.einnews.com/article/5120893...,E


Load the parquet dataframe with attributes of each author

In [19]:
speaker_attributes_updated = pd.read_parquet("data/speaker_attributes_updated.parquet")


In [20]:
speaker_attributes_updated

,aliases,date_of_birth,nationality,gender,lastrevid,ethnic_group,US_congress_bio_ID,occupation,party,academic_degree,id,label,candidacy,type,religion
0,"[Washington, President Washington, G. Washingt...",[+1732-02-22T00:00:00Z],"[Great Britain, United States of America]",[male],1395141751,None,W000178,"[politician, military officer, farmer, cartogr...",[independent politician],None,Q23,George Washington,"[1792 United States presidential election, 178...",item,[Episcopal Church]
1,"[Douglas Noel Adams, Douglas Noël Adams, Dougl...",[+1952-03-11T00:00:00Z],[United Kingdom],[male],1395737157,[White British],None,"[playwright, screenwriter, novelist, children'...",None,None,Q42,Douglas Adams,None,item,None
2,"[Paul Marie Ghislain Otlet, Paul Marie Otlet]",[+1868-08-23T00:00:00Z],[Belgium],[male],1380367296,None,None,"[writer, lawyer, librarian, information scient...",None,None,Q1868,Paul Otlet,None,item,None
3,"[George Walker Bush, Bush Jr., Dubya, GWB, Bus...",[+1946-07-06T00:00:00Z],[United States of America],[male],1395142029,None,None,"[politician, motivational speaker, autobiograp...",[Republican Party],None,Q207,George W. Bush,"[2000 United States presidential election, 200...",item,"[United Methodist Church, Episcopal Church, Me..."
4,"[Velázquez, Diego Rodríguez de Silva y Velázqu...",[+1599-06-06T00:00:00Z],[Spain],[male],1391704596,None,None,[painter],None,None,Q297,Diego Velázquez,None,item,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9055976,[Barker Howard],None,[United States of America],[male],1397399351,None,None,[politician],None,None,Q106406560,Barker B. Howard,None,item,None
9055977,[Charles Macomber],None,[United States of America],[male],1397399471,None,None,[politician],None,None,Q106406571,Charles H. Macomber,None,item,None
9055978,None,[+1848-04-01T00:00:00Z],None,[female],1397399751,None,None,None,None,None,Q106406588,Dina David,None,item,None
9055979,None,[+1899-03-18T00:00:00Z],None,[female],1397399799,None,None,None,None,None,Q106406593,Irma Dexinger,None,item,None


### Merge Chunk with Speaker Attributtes

For an arbitrary chunk add a column with the row corresponding to the speaker

In [17]:
chunk = df_Trump[:100]

chunk

,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
12,2018-07-16-000103,[ Ensuring ] the orchestrating and timing of M...,Corey Lewandowski,[Q20740735],2018-07-16 14:05:34,2,"[[Corey Lewandowski, 0.7179], [None, 0.2754], ...",[http://www.theweek.co.uk/95082/donald-trump-s...,E
66,2018-05-09-001003,300-plus years of them cold shoulders... Obama...,Charlamagne Tha God,[Q16203002],2018-05-09 11:00:00,1,"[[Charlamagne Tha God, 0.4806], [None, 0.2924]...",[https://www.portlandmercury.com/music/2018/05...,E
212,2018-08-02-002115,A politically connected contractor made a $500...,None,[],2018-08-02 06:24:59,1,"[[None, 0.7942], [President Trump, 0.1406], [K...",[https://sunlightfoundation.com/2018/08/02/tod...,E
264,2018-08-19-001084,A Wider Danger: Trump's Troubling Attacks on J...,None,[],2018-08-19 02:05:39,3,"[[None, 0.9052], [Queen Elizabeth II, 0.0948]]","[http://www.vnews.com/Forum-Aug-19-19549635, h...",E
333,2018-09-26-004202,"After the commercial, she says, `They told me ...",None,[],2018-09-26 00:00:00,2,"[[None, 0.4595], [Tom Arnold, 0.3248], [Julie ...",[http://feeds.foxnews.com/~r/foxnews/entertain...,E
...,...,...,...,...,...,...,...,...,...
10021,2018-12-27-064793,These reporters can't even begin a news accoun...,None,[],2018-12-27 17:47:14,37,"[[None, 0.905], [Melania Trump, 0.0594], [Jame...",[https://www.newsbusters.org/blogs/nb/curtis-h...,E
10098,2018-05-02-131151,"They elected Donald Trump, who really understa...",Rachel Campos-Duffy,[Q2267027],2018-05-02 19:45:19,1,"[[Rachel Campos-Duffy, 0.4752], [None, 0.3776]...",[http://insider.foxnews.com/2018/05/02/rachel-...,E
10331,2018-05-10-133645,This day is something that we didn't think we'...,Phil Murphy,"[Q260042, Q3378818, Q7182210, Q7182211]",2018-05-10 01:19:21,7,"[[Phil Murphy, 0.5115], [None, 0.4885]]",[https://www.tapinto.net/towns/newark/articles...,E
10377,2018-09-26-118017,This is a classic espionage `wilderness of mir...,Mark Steyn,[Q332357],2018-09-26 10:02:49,2,"[[Mark Steyn, 0.8879], [None, 0.0591], [Chris ...",[https://patriotpost.us/articles/58450-wednesd...,E


Add the id of the special_attribute row to each quote

In [11]:
def search_author_index(QID, authors):
    res = authors[authors["id"] == QID]
    if res.empty:
        return -1
    return res.index[0]

In [12]:
def assign_value(chunk, index, key, QID, authors):
    chunk.at[index, key] = search_author_index(QID, authors)

In [31]:
def add_author_id(chunk, authors):
    import concurrent.futures
    executor = concurrent.futures.ThreadPoolExecutor(30)
    if "authorId" not in chunk: 
        chunk.insert(chunk.shape[1], "authorId", -1)
    for index, row in chunk.iterrows():
        executor.submit(assign_value, chunk, index, "authorId", row.qids[0], authors)
    executor.shutdown()
    return chunk[chunk["authorId"] != -1]

In [32]:
import time

start = time.time()

processed_chunk = add_author_id(chunk, speaker_attributes_updated)

end = time.time()

print((end - start))

processed_chunk

quoteID                                           2018-07-16-000103
quotation         [ Ensuring ] the orchestrating and timing of M...
speaker                                           Corey Lewandowski
qids                                                    [Q20740735]
date                                            2018-07-16 14:05:34
numOccurrences                                                    2
probas            [[Corey Lewandowski, 0.7179], [None, 0.2754], ...
urls              [http://www.theweek.co.uk/95082/donald-trump-s...
phase                                                             E
authorId                                                    7154188
Name: 12, dtype: object
quoteID                                           2018-05-09-001003
quotation         300-plus years of them cold shoulders... Obama...
speaker                                         Charlamagne Tha God
qids                                                    [Q16203002]
date                    

IndexError: list index out of range

### Random Sampling

Get a random sampling from the data, the distribution should be the same and the dataset will be manageable given the computation restrictions

In [170]:
def random_sample(dataset, sample_size):
    return dataset.sample(n=sample_size)

### After 

In [ ]:
df_Trump = pd.read_csv('df_Trump_cleaned.csv', header =0, index_col=0, parse_dates=True, squeeze = True )
df_Trump.index = df_Trump.index.map(lambda x: str(x)[:-7]) #we transform the quote_ID in a format date
df_Trump.to_csv('df_Trumps_with_dates.csv')

In [ ]:
df_Trump = pd.read_csv('df_Trumps_with_dates.csv',header =0, index_col=0, parse_dates=True, squeeze = True )

In [ ]:
df_Trump.head(10) #as we can see now we have the dates

In [ ]:
# we plot the Trump timeseries of the number of quotes over the past few years
plt.rcParams["figure.figsize"] = (18,6)
ax = df_Trump.plot()
plt.title('timeseries of the number of occurences of quotes referring to Donald Trump')
plt.xlabel('time')
plt.show()

The time serie is quite interesting, as we can see there are some peaks in it, one big peak at the end of 2017. In milestone 3 we will try to link these peaks with the political events in his career

### We could try to filter the dataset finding just the quotes in the two months before and after the elections of november 2016 in order to understand if this event correspond to a major number of quotes referring to Trump

In [ ]:
df_Trump_2_months_before = df_Trump[('2016-11-09'>=df_Trump.index) & (df_Trump.index >= '2016-09-01')] 

In [ ]:
df_Trump_2_months_before

In [ ]:
# we plot the Trump timeseries of the number of quotes over the past two months before the elections
plt.rcParams["figure.figsize"] = (12,6)
ax = df_Trump_2_months_before.plot()
plt.title('timeseries of the number of occurences of quotes referring to Donald Trump two months before the elections')
plt.xlabel('time')
plt.show()

In [ ]:
df_Trump_2_months_after = df_Trump[('2016-11-09'<=df_Trump.index) & (df_Trump.index <= '2017-01-01')] 

In [ ]:
df_Trump_2_months_after

In [ ]:
# we plot the Trump timeseries of the number of quotes over the past two months before the elections
plt.rcParams["figure.figsize"] = (12,6)
ax = df_Trump_2_months_after.plot()
plt.title('timeseries of the number of occurences of quotes referring to Donald Trump two months before the elections')
plt.xlabel('time')
plt.show()

In [ ]:
def major_speakers(df, politician):
    print('the people who are speaking the most about '+politician+' are\n', df['speaker'].value_counts()[:10].index.tolist())
    df['speaker'].value_counts()[:10].plot(kind='bar', logy=True)
    plt.title('people who are speaking the most about '+politician)
    plt.ylabel('number of quotes')
    plt.show()

In [ ]:
major_speakers(df_Trump, 'Trump')

As we can see Hillary Clinton and Joe Biden are present in the list. This is quite obvious since they have been the two contenders at the last political elections and it is clear that they had to quote Trump a lot in order to discredit his opinion.

In [ ]:
del df_Trump # we don't want to store it in memory

### we do the same for Clinton

In [ ]:
df_Clinton = pd.read_csv('df_Clinton_cleaned.csv',header =0, index_col=0, parse_dates=True, squeeze =True )

In [ ]:
df_Clinton.index = df_Clinton.index.map(lambda x: str(x)[:-7]) # we transform the quote_id in a format date
df_Clinton.to_csv('df_Clinton_with_dates.csv') #checkpoint

In [ ]:
import matplotlib.ticker as plticker

plt.rcParams["figure.figsize"] = (18,6)
ax = df_Clinton.plot()
plt.title('timeseries of the number of occurences of quotes referring to Hillary Clinton')
plt.xlabel('time')
plt.show()

As we can see one of the peaks is the one corresponding to the elections of november 2016

### It could be interseting to know who are the people who quote Clinton the most, we will try to understand it

In [ ]:
major_speakers(df_Clinton, 'Clinton')

As we can see Trump is quoting a lot Clinton, this could be related to the fact that a big part of his political campaign of 2016 was based on discrediting his contender (Clinton)